# daemon讲解

### 目录
- 基本概念
- 基本原理 
- PEP 4134 设计
- python-daemon库
- python-daemon实例
- 附录 A: 一个python daemon的实现
- 附录 B: 其他daemon的实现


## 基本概念 

### 进程

进程是程序的一个运行实例，也是执行程序的过程。同一个程序可以执行多次，每次都对应一个进程。在每次执行程序时，进程开辟独立内存空间、IO接口。操作系统的一个重要功能就是进程管理，其功能主要是为进程分配内存空间，管理进程的相关信息等等。

在Linux中，可用$ps命令来查询正在运行的进程。关于ps命令可参加：https://blog.csdn.net/freeking101/article/details/53444530

### fork() 

#### fork 处理过程

在类Unix系统中，操作系统提供了fork()系统调用，用于创建新的进程。与其他系统调用不同，fork()有如下特性：  
- 参数：无参数
- 返回值
    - 如果成功创建一个子进程，对于父进程来说返回子进程ID
    - 如果成功创建一个子进程，对于子进程来说返回值为0
    - 如果为-1表示创建失败（比如内存溢出等，无法创建子进程）
- 在fork()返回后，父、子进程开始从下一条指令开始执行。

In [45]:
# 实例： 说明fork的父、子进程都是从fork()函数的下一句开始，并行执行。 多层树型结构
import os
#print('Process %s start. ppid=%s'%(os.getpid(), os.getppid()))
pid = os.fork()
pid = os.fork()
pid = os.fork()
pid = os.fork()
# how many time to print?  指数级增长：2**n 次
print('(%s, %s<-%s)'%(pid, os.getpid(), os.getppid()))

(13925, 6334<-6098)
(13926, 13904<-6334)
(13917, 13912<-6334)
(13916, 13908<-13905)
(13927, 13905<-6334)
(0, 13927<-13905)
(0, 13916<-13908)
(13930, 13915<-13907)
(0, 13917<-13912)
(0, 13926<-13904)
(13937, 13907<-13904)
(0, 13925<-6334)
(13942, 13924<-13904)
(0, 13930<-13915)
(0, 13937<-13907)
(0, 13942<-13924)


In [40]:
# 实例： 同一个父进程，创建多个不同的子进程。 双层树型结构
import os
#print('Process %s start. ppid=%s'%(os.getpid(), os.getppid()))
pid = os.fork()
if (pid > 0):
    os.fork()
    print('In perent( new child:%s, %s<-%s)'%(pid, os.getpid(), os.getppid()))
else:
    print('In child(%s, %s<-%s)'%(pid, os.getpid(), os.getppid()))

In perent( new child:13610, 6334<-6098)
In child(0, 13610<-6334)
In perent( new child:13610, 13611<-6334)


#### fork 子进程资源
fork出的子进程继承了父进程下面这些属性：
- uid,gid,euid,egid
- 附加组id(sgid,supplementary group id) //sgid引入原因是有时候希望这个用户属于多个其他部门，这些其他部门的gid就是sgid
- 进程组id,会话id
- SUID标记和SGID标记
- 控制终端
- 当前工作目录/根目录
- 文件创建时的umask
- 文件描述符的文件标志(close-on-exec)
- 信号屏蔽和处理
- 存储映射
- 资源限制  

未被继承的属性有：
- pid不同
- 进程时间被清空
- 文件锁没有继承
- 未处理信号被清空

https://cloud.tencent.com/developer/article/1012442
https://blog.csdn.net/u013210620/article/details/78710532
https://www.cnblogs.com/qianchengprogram/p/6605675.html
https://www.jianshu.com/p/e3f3d49093ca
https://www.jb51.net/article/128316.htm
https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001431927781401bb47ccf187b24c3b955157bb12c5882d000

### 多线程（multithread）

### fock 进程